In [ ]:
# 🛠️ Task 3: Crowd Detection - All CSV Results, Limit 100 Frames Saved

# STEP 1: Install required packages
#!pip install ultralytics opencv-python pandas

# STEP 2: Import libraries
from ultralytics import YOLO
import cv2
import pandas as pd
import os
import shutil
from google.colab import files

# STEP 3: Upload video file
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

# STEP 4: Load YOLOv8 model
model = YOLO('yolov8n.pt')

# STEP 5: Create output directory
output_dir = "crowd_frames_100"
os.makedirs(output_dir, exist_ok=True)

# STEP 6: Set saved frame limit
MAX_SAVED_FRAMES = 100
saved_frame_count = 0

# STEP 7: Process entire video and detect crowd
cap = cv2.VideoCapture(video_path)
frame_count = 0
crowd_log = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    # ✅ Filter for 'person' class only (class 0) with confidence > 0.5
    persons = [
        box for box in results.boxes
        if int(box.cls[0]) == 0 and float(box.conf[0]) > 0.5
    ]

    # Save frame if crowd (3+ persons) is detected
    if len(persons) >= 3:
        # Save up to 100 frames with bounding boxes
        if saved_frame_count < MAX_SAVED_FRAMES:
            for box in persons:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            frame_path = f"{output_dir}/frame_{frame_count}.jpg"
            cv2.imwrite(frame_path, frame)
            saved_frame_count += 1

        # Always log the frame info for CSV
        crowd_log.append((frame_count, len(persons)))

    frame_count += 1

cap.release()

# STEP 8: Analyze crowd frames (full log, not limited)
df = pd.DataFrame(crowd_log, columns=["frame", "person_count"])
df["frame_diff"] = df["frame"].diff().fillna(1)
df["group_id"] = (df["frame_diff"] != 1).cumsum()

crowds = []
for group_id, group in df.groupby("group_id"):
    if len(group) >= 10:
        start = int(group.iloc[0]["frame"])
        end = int(group.iloc[-1]["frame"])
        avg = int(group["person_count"].mean())
        crowds.append((start, end, avg))

# STEP 9: Save CSV and ZIP frames
result_df = pd.DataFrame(crowds, columns=["start_frame", "end_frame", "avg_person_count"])
result_df.to_csv("crowd_results.csv", index=False)

# ZIP the crowd frames (only up to 100 saved)
shutil.make_archive("crowd_frames_100", 'zip', output_dir)

# STEP 10: Download results
print("✅ Done! Downloading files...")
files.download("crowd_results.csv")
#files.download("crowd_frames_100.zip")


Saving dataset_video.mp4 to dataset_video.mp4


100%|██████████| 6.25M/6.25M [00:00<00:00, 77.7MB/s]



0: 384x640 37 persons, 2 birds, 413.2ms
Speed: 21.5ms preprocess, 413.2ms inference, 43.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 3 birds, 173.8ms
Speed: 14.2ms preprocess, 173.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 35 persons, 3 birds, 155.3ms
Speed: 4.4ms preprocess, 155.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 persons, 3 birds, 175.7ms
Speed: 4.7ms preprocess, 175.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 persons, 2 birds, 148.8ms
Speed: 5.8ms preprocess, 148.8ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 2 birds, 136.1ms
Speed: 10.2ms preprocess, 136.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 persons, 2 birds, 153.9ms
Speed: 4.0ms preprocess, 153.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 persons,

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("crowd_frames_100.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>